In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/",one_hot=True)

# X의 첫번째 차원인 None은 입력 데이터의 개수입니다. 그리고 마지막 차원
# 인 1은 특징의 개수로, MNIST데이터는 회색조 이미지라 채널에 색상이 한 개
# 뿐이므로 1을 사용하였습니다. 그리고 출력값인 10개의 분류값으로 정의합니다.
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)


# 컨볼루션계층
# 입력층 X와 첫 번째 계층의 가중치 W1을 가지고, 오른쪽과 아래쪽으로 한칸
# 씩 움직이는 32개의 커널을 가진 컨볼루션 계층을 만들겟다는 코드입니다. 여기
# 서 padding='SAME'은 커널 슬라이딩 시 이미지의 가장 외곽에서 한 칸 밖으로
# 움직이는 옵션입니다. 이렇게 하면 이미지의 테두리까지도 조금 더 정확하게 평
# 가할 수 있습니다.
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)

# 풀링계층
# 앞서 만든 컨볼루션 계층을 입력층으로 사용하고, 커널 크기를 2x2로 하는 풀링
# 계층을 만듭니다. strides=[1, 2, 2, 1]값은 슬라이딩 시 두 칸씩
# 움직이겟다는 옵션입니다.
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')


# 3x3크기의 커널 64개로 구성한 컨볼루션 계층과 2x2크기의 풀링 계층으로 구성하였습니다.
# 두번째 컨볼루션 계층의 커널인 W2 변수의 구성은 [3, 3, 32, 64] 입니다.
# 여기에서 32는 앞서 구성한 첫 번째 컨볼루션 계층의 커널 개수입니다.
# 이것은 출력층의 개수이며 또한 첫 번째 컨볼루션 계층이 찾아낸 개수라고 할 수 있겟습니다.
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# 먼저 10개의 분류는 1차원 배열이므로 차원을 줄이는 단계를 거쳐야 합니다.
# 직전의 풀링 계층 크기가 7x7x64이므로, 먼저 tf.reshape함수를 이용해 7x7x64 크기의 1차원 계층으로 만들고,
# 이 배열 전체를 최종 출력값의 중간 단계인 256개의 뉴런으로 연결하는 신경망을 만들어줍니다.
W3 = tf.Variable(tf.random_normal([7 * 7 * 64, 256], stddev=0.01))
L3 = tf.reshape(L2, [-1, 7 * 7 * 64])
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)
L3 = tf.nn.dropout(L3, keep_prob)


# 이제 모델구성의 마지막으로 직전의 은닉층인 L3의 출력값 256개를 받아 최종 출력값인 0~9레이블을 갖는 10개의 출력값을 만듭니다.
W4 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L3, W4)


cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)
#optimizer = tf.train.EMSPropOptimizer(0.001, 0.9).minimize(cost)

In [ ]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(15):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        
        _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, Y: batch_ys, keep_prob: 0.7})
        total_cost += cost_val
        
    print('Epoch:', '%04d' % (epoch + 1), 'Avg. cost =' '{:.3f}'.format(total_cost / total_batch))
    
print('최적화 완료!')

is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

print('정확도:', sess.run(accuracy, feed_dict={X: mnist.test.images.reshape(-1, 28, 28, 1),
                                            Y: mnist.test.labels, keep_prob: 1 }))